In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import csv
import numpy

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leosh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
url_meta = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc'
page_meta = requests.get(url_meta, headers = headers)
soup_meta = BeautifulSoup(page_meta.content, "html.parser")

In [3]:
container_meta = soup_meta.find_all('td', class_ = 'clamp-summary-wrap')
print(type(container_meta))

<class 'bs4.element.ResultSet'>


In [4]:
movie_names_meta = []
movies_meta = container_meta[:50] #here get the top 50 movies we want
for movie_meta in movies_meta:
    name_meta = movie_meta.find('h3').text
    movie_names_meta.append(name_meta)
print(movie_names_meta)

['Citizen Kane', 'The Godfather', 'Rear Window', 'Casablanca', 'Boyhood', 'Three Colors: Red', 'Vertigo', 'Notorious', "Singin' in the Rain", 'City Lights', 'Moonlight', 'Intolerance', 'Pinocchio', 'Touch of Evil', 'The Treasure of the Sierra Madre', "Pan's Labyrinth", 'Some Like It Hot', 'North by Northwest', 'Hoop Dreams', 'Rashomon', 'All About Eve', 'Jules and Jim', 'The Wild Bunch', 'My Left Foot', 'The Third Man', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Nomadland', 'Gone with the Wind', '4 Months, 3 Weeks and 2 Days', 'Psycho', 'Battleship Potemkin', 'A Streetcar Named Desire', 'American Graffiti', 'Dumbo', 'Roma', 'Ran', 'The Shop Around the Corner', '12 Angry Men', 'Manchester by the Sea', "Rosemary's Baby", 'The Maltese Falcon', '12 Years a Slave', 'Killer of Sheep', 'Rocks', 'Nashville', 'Ratatouille', 'Parasite', "Don't Look Now", 'The Grapes of Wrath', 'Children of Paradise (1945)']


In [5]:
#now need to click into each movie name in order to see detailed information
#so need to extract all the urls

movie_links_meta = []
movies_meta = container_meta[:50]
for movie_meta in movies_meta:
    tag_meta = movie_meta.find('a', class_ = 'title')
    link_meta = tag_meta.get('href', None)
    movie_links_meta.append(link_meta)
print(movie_links_meta)

['/movie/citizen-kane', '/movie/the-godfather', '/movie/rear-window', '/movie/casablanca', '/movie/boyhood', '/movie/three-colors-red', '/movie/vertigo', '/movie/notorious-1946', '/movie/singin-in-the-rain', '/movie/city-lights', '/movie/moonlight-2016', '/movie/intolerance', '/movie/pinocchio-1940', '/movie/touch-of-evil', '/movie/the-treasure-of-the-sierra-madre', '/movie/pans-labyrinth', '/movie/some-like-it-hot', '/movie/north-by-northwest', '/movie/hoop-dreams', '/movie/rashomon', '/movie/all-about-eve', '/movie/jules-and-jim', '/movie/the-wild-bunch', '/movie/my-left-foot', '/movie/the-third-man', '/movie/dr-strangelove-or-how-i-learned-to-stop-worrying-and-love-the-bomb', '/movie/nomadland', '/movie/gone-with-the-wind', '/movie/4-months-3-weeks-and-2-days', '/movie/psycho-1960', '/movie/battleship-potemkin', '/movie/a-streetcar-named-desire', '/movie/american-graffiti', '/movie/dumbo-1941', '/movie/roma', '/movie/ran', '/movie/the-shop-around-the-corner', '/movie/12-angry-men', 

In [6]:
for i in range(len(movie_links_meta)):
    movie_links_meta[i] = 'https://www.metacritic.com'+ movie_links_meta[i]
print(movie_links_meta)

['https://www.metacritic.com/movie/citizen-kane', 'https://www.metacritic.com/movie/the-godfather', 'https://www.metacritic.com/movie/rear-window', 'https://www.metacritic.com/movie/casablanca', 'https://www.metacritic.com/movie/boyhood', 'https://www.metacritic.com/movie/three-colors-red', 'https://www.metacritic.com/movie/vertigo', 'https://www.metacritic.com/movie/notorious-1946', 'https://www.metacritic.com/movie/singin-in-the-rain', 'https://www.metacritic.com/movie/city-lights', 'https://www.metacritic.com/movie/moonlight-2016', 'https://www.metacritic.com/movie/intolerance', 'https://www.metacritic.com/movie/pinocchio-1940', 'https://www.metacritic.com/movie/touch-of-evil', 'https://www.metacritic.com/movie/the-treasure-of-the-sierra-madre', 'https://www.metacritic.com/movie/pans-labyrinth', 'https://www.metacritic.com/movie/some-like-it-hot', 'https://www.metacritic.com/movie/north-by-northwest', 'https://www.metacritic.com/movie/hoop-dreams', 'https://www.metacritic.com/movie/

In [7]:
#here find the user reviews of the movie, there are 2 reviews critic and user, we will extract user reviews
# found the genre of each movie and stored in list format

review_links_meta = []
genre_lst_meta = []
for i in movie_links_meta:
    page_meta = requests.get(i, headers = headers)
    soup_meta = BeautifulSoup(page_meta.content, "html.parser")
    tag_meta = soup_meta.find_all('a', class_ = 'see_all boxed oswald') #here we are using find_all, so it will return a list
    user_review_tag_meta = tag_meta[1] #we only want the second one
    review_link_meta = user_review_tag_meta.get('href', None)
    review_links_meta.append(review_link_meta)
    
    #genre
    m_page_meta = requests.get(i, headers = headers)
    m_soup_meta = BeautifulSoup(m_page_meta.content, 'lxml')
    genre_meta = m_soup_meta.find('div', class_ = 'genres').text
    genre_meta = re.sub('Genre\(s\):','',genre_meta).lower()
    genre_meta = re.sub('\s+','',genre_meta).split(',')
    genre_lst_meta.append(genre_meta)
    
print(review_links_meta)
print(genre_lst_meta)
print(len(genre_lst_meta))

['/movie/citizen-kane-1941/user-reviews', '/movie/the-godfather/user-reviews', '/movie/rear-window/user-reviews', '/movie/casablanca/user-reviews', '/movie/boyhood/user-reviews', '/movie/three-colors-red/user-reviews', '/movie/vertigo-1958/user-reviews', '/movie/notorious-1946/user-reviews', '/movie/singin-in-the-rain/user-reviews', '/movie/city-lights/user-reviews', '/movie/moonlight-2016/user-reviews', '/movie/intolerance/user-reviews', '/movie/pinocchio-1940/user-reviews', '/movie/touch-of-evil/user-reviews', '/movie/the-treasure-of-the-sierra-madre/user-reviews', '/movie/pans-labyrinth/user-reviews', '/movie/some-like-it-hot/user-reviews', '/movie/north-by-northwest/user-reviews', '/movie/hoop-dreams/user-reviews', '/movie/rashomon/user-reviews', '/movie/all-about-eve/user-reviews', '/movie/jules-and-jim/user-reviews', '/movie/the-wild-bunch/user-reviews', '/movie/my-left-foot/user-reviews', '/movie/the-third-man/user-reviews', '/movie/dr-strangelove-or-how-i-learned-to-stop-worryi

In [8]:
# here found the user reviews and the list of words excluding the stopwords in the wordlist_meta

tag_lst_meta = []
wordlist_meta = []
n_meta = 0
for i in range(len(review_links_meta)):
    if not review_links_meta[i].endswith('user-reviews'):
        continue
    else:
        review_page_meta = requests.get('https://www.metacritic.com'+review_links_meta[i], headers = headers)
        soup_meta = BeautifulSoup(review_page_meta.content, "html.parser")
        tag_meta_p = soup_meta.find_all('div', class_ = 'summary') #find_all to return a list
        tag_meta_f = soup_meta.find('div', class_ = 'review_body')
        tag_meta_r = re.sub("<.?>", "", tag_meta_f.text)
        tag_meta_r = (tag_meta_r.strip()).lower()
        tag_meta = re.sub(r'[^\w\s]', '', tag_meta_r)
        text_token_meta = word_tokenize(tag_meta)
        review3_meta = [word for word in text_token_meta if not word in stopwords.words()]
        n_meta = n_meta + 1
        print('Movie', n_meta)
        top_50_meta = tag_meta[:50]
        print('Total user reviews:', len(top_50_meta))
        print(len(top_50_meta))  
        print(tag_meta)
        tag_lst_meta.append(tag_meta)
        wordlist_meta += review3_meta  #created the list of words without stopwords

print(wordlist_meta)
for _ in range(len(tag_lst_meta), 50):
    tag_lst_meta.append('Null')

Movie 1
Total user reviews: 50
50
its generally a good movie and pretty watchable but in no way is it the greatest movie of alltime in truth probably not even in the top 1000
Movie 2
Total user reviews: 50
50
like babe ruth is to baseball like john steinbeck is to writing their is no doubt about it the godfather is the greatest film of all time never before have we seen performances so stellar by marlon brando al pacino james caan robertlike babe ruth is to baseball like john steinbeck is to writing their is no doubt about it the godfather is the greatest film of all time never before have we seen performances so stellar by marlon brando al pacino james caan robert duvall and everyone else the godfather has little life lessons everywhere along with an engaging plot that just completes the most critically acclaimed film of all time the godfather is francis ford coppolas masterpiece expand
Movie 3
Total user reviews: 50
50
jeff jefferies is a photographer who likes to spy on the lives of

Movie 14
Total user reviews: 50
50
interesting title considering the studio recut it in a fight with welles that ended in them not wanting to submit it for awards and him never working for a studio again  also interesting is that welles character mirrored his life at thatinteresting title considering the studio recut it in a fight with welles that ended in them not wanting to submit it for awards and him never working for a studio again  also interesting is that welles character mirrored his life at that time  i saw the 1998 version reconstructed from welles notes  it is fantastic his 2nd best movie just have to get past charlton heston as a mexican expand
Movie 15
Total user reviews: 50
50
remember that treasure of sierra madre was a movie made under a context where gaves value to text and plot which doesnt an exception here beyond fit and welltied form here we got some demonstrations of loyalty corruption poverty andremember that treasure of sierra madre was a movie made under a cont

Movie 33
Total user reviews: 50
50
one of my favorites ever all that happens on that day changes everyones life each and every single scene is important everything beautifully portrayed cute carol already stole my heart no pedo im also 15
Movie 34
Total user reviews: 50
50
after the disppointing failure of fantasia dumbo hit the big screen when i was kid i love it as an adult i still love it the opening tune sounds catchy the songs wooing their way on everybodys heads the animation just like the othersafter the disppointing failure of fantasia dumbo hit the big screen when i was kid i love it as an adult i still love it the opening tune sounds catchy the songs wooing their way on everybodys heads the animation just like the others is well drawn and my favorite character is got to be the crows well yeah i know people think theyre racist but there only in the movie for about 10 minutes but theyre still funny and also i like the clowns as well theres a scene where dumbos mother goes crazy

Movie 44
Total user reviews: 50
50
i have no idea how its possible to fully flesh out all of these characters in a film with such a freeroaming plot i also have absolutely no clue how every plotline that concerns a specific character or a specific group of charactersi have no idea how its possible to fully flesh out all of these characters in a film with such a freeroaming plot i also have absolutely no clue how every plotline that concerns a specific character or a specific group of characters kinda intertwined or rather formed altogether a complete and very impactful picture by the endit took me some time to get invested but once i found an interesting character i gradually became fully immersed in what was going on also i have to admit that i wasnt constantly or equally engaged with each storyline but that will definitely change upon rewatchthe songs are some of the best ive ever heard in a film the welldeserved oscarwinning song im easy is simply brilliant its quite a refreshing ex

IMDB Reviews

In [9]:
url_imdb = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
page_imdb = requests.get(url_imdb, headers = headers)
soup_imdb = BeautifulSoup(page_imdb.content, "html.parser")

In [10]:
container_imdb = soup_imdb.find_all('td', class_ = 'titleColumn')
print(type(container_imdb))

<class 'bs4.element.ResultSet'>


In [11]:
movie_names_imdb = []
movies_imdb = container_imdb[:50] #here got the top 50 movies we want
for movie_imdb in movies_imdb:
    name_imdb = movie_imdb.find('a').text
    movie_names_imdb.append(name_imdb)
print(movie_names_imdb)

['The Shawshank Redemption', 'The Godfather', 'The Godfather: Part II', 'The Dark Knight', '12 Angry Men', "Schindler's List", 'The Lord of the Rings: The Return of the King', 'Pulp Fiction', 'The Good, the Bad and the Ugly', 'The Lord of the Rings: The Fellowship of the Ring', 'Fight Club', 'Forrest Gump', 'Inception', 'The Lord of the Rings: The Two Towers', 'Star Wars: Episode V - The Empire Strikes Back', 'The Matrix', 'Goodfellas', "One Flew Over the Cuckoo's Nest", 'Seven Samurai', 'Se7en', 'Life Is Beautiful', 'City of God', 'The Silence of the Lambs', "It's a Wonderful Life", 'Star Wars: Episode IV - A New Hope', 'Saving Private Ryan', 'Spirited Away', 'The Green Mile', 'Interstellar', 'Parasite', 'Léon: The Professional', 'The Usual Suspects', 'Hara-Kiri', 'The Lion King', 'The Pianist', 'Back to the Future', 'Terminator 2: Judgment Day', 'American History X', 'Modern Times', 'Psycho', 'Gladiator', 'City Lights', 'The Departed', 'The Intouchables', 'Whiplash', 'The Prestige', 

In [12]:
#now need to click into each movie name in order to see detailed information
#so need to extract all the urls

movie_links_imdb = []
movies_imdb = container_imdb[:50]
for movie_imdb in movies_imdb:
    tag_imdb = movie_imdb.find('a')
    link_imdb = tag_imdb.get('href', None)
    movie_links_imdb.append(link_imdb)
print(movie_links_imdb)

['/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1', '/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2', '/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3', '/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_4', '/title/tt0050083/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_5', '/title/tt0108052/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5P

In [13]:
for i in range(len(movie_links_imdb)):
    movie_links_imdb[i] = 'https://www.imdb.com'+ movie_links_imdb[i]
print(movie_links_imdb)

['https://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1', 'https://www.imdb.com/title/tt0068646/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_2', 'https://www.imdb.com/title/tt0071562/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_3', 'https://www.imdb.com/title/tt0468569/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_4', 'https://www.imdb.com/title/tt0050083/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=NJV35EW498MGBY6QG5PA&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_5', 'https://www.i

In [14]:
# found the genre of each movie and stored in list format

genre_lst_imdb = []
for i in movie_links_imdb:
    m_page_imdb = requests.get(i, headers = headers)
    m_soup_imdb = BeautifulSoup(m_page_imdb.content, 'lxml')
    genre = m_soup_imdb.find('div', class_ = 'subtext')
    genre_imdb_p = genre.find('a').text.lower().strip()
    #genre_meta_f = re.sub('Genre\(s\):','',genre_meta_p).lower()
    genre_imdb = re.sub('\s+','',genre_imdb_p).split(',')
    genre_lst_imdb.append(genre_imdb)
    
print(genre_lst_imdb)

[['drama'], ['crime'], ['crime'], ['action'], ['crime'], ['biography'], ['action'], ['crime'], ['western'], ['action'], ['drama'], ['drama'], ['action'], ['action'], ['action'], ['action'], ['biography'], ['drama'], ['action'], ['crime'], ['comedy'], ['crime'], ['crime'], ['drama'], ['action'], ['drama'], ['animation'], ['crime'], ['adventure'], ['comedy'], ['action'], ['crime'], ['action'], ['animation'], ['biography'], ['adventure'], ['action'], ['drama'], ['comedy'], ['horror'], ['action'], ['comedy'], ['crime'], ['biography'], ['drama'], ['drama'], ['animation'], ['western'], ['biography'], ['drama']]


In [15]:
#found the link of the reviews of each movie

review_links_imdb = []
for i in movie_links_imdb:
    soup_imdb = BeautifulSoup(page_imdb.content, "html.parser")
    text_imdb = i
    subtext_imdb = re.search('^https:\/.*\/',text_imdb).group(0)
    review_link_imdb = subtext_imdb + 'reviews?ref_=tt_urv'
    review_links_imdb.append(review_link_imdb)
print(review_links_imdb)

['https://www.imdb.com/title/tt0111161/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0068646/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0071562/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0468569/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0050083/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0108052/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0167260/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0110912/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0060196/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0120737/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0137523/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0109830/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt1375666/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0167261/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0080684/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0133093/reviews?ref_=tt_urv', 'https://www.imdb.com/title/tt0099685/r

In [16]:
# here found the user reviews and the list of words excluding the stopwords in the wordlist_imdb

tag_list_imdb = []
wordlist_imdb = []
n_imdb = 0
for i in range(len(review_links_imdb)):
    review_page_imdb = requests.get(review_links_imdb[i], headers = headers)
    soup_imdb = BeautifulSoup(review_page_imdb.content, "html.parser")
    tag_p_imdb = soup_imdb.find_all('div', class_ = 'content')               #find_all to return a list
    tag_imdb_f = soup_imdb.find('div', class_ = 'text show-more__control')
    tag_imdb_r = re.sub("<.?>", "", tag_imdb_f.text)
    tag_imdb_r = (tag_imdb_r.strip()).lower()
    tag_imdb = re.sub(r'[^\w\s]', '', tag_imdb_r)
    text_token_imdb = word_tokenize(tag_imdb)
    review3_imdb = [word for word in text_token_imdb if not word in stopwords.words()] 
    n_imdb = n_imdb + 1
    print('Movie', n_imdb)
    top_50_imdb = tag_imdb[:50]
    print('Total user reviews:', len(top_50_imdb))
    print(len(top_50_imdb)) 
    print(tag_imdb)
    tag_list_imdb.append(tag_imdb)
    wordlist_imdb += review3_imdb
    
print(wordlist_imdb)

Movie 1
Total user reviews: 50
50
the shawshank redemption is written and directed by frank darabont it is an adaptation of the stephen king novella rita hayworth and shawshank redemption starring tim robbins and morgan freeman the film portrays the story of andy dufresne robbins a banker who is sentenced to two life sentences at shawshank state prison for apparently murdering his wife and her lover andy finds it tough going but finds solace in the friendship he forms with fellow inmate ellis red redding freeman while things start to pick up when the warden finds andy a prison job more befitting his talents as a banker however the arrival of another inmate is going to vastly change things for all of themthere was no fanfare or bunting put out for the release of the film back in 94 with a title that didnt give much inkling to anyone about what it was about and with columbia pictures unsure how to market it shawshank redemption barely registered at the box office however come academy awa

Movie 4
Total user reviews: 50
50
confidently directed dark brooding and packed with impressive action sequences and a complex story the dark knight includes a careerdefining turn from heath ledger as well as other oscar worthy performances tdk remains not only the best batman movie but comic book movie ever created
Movie 5
Total user reviews: 50
50
an excellent courtroom drama with a unique twist instead of following the trial itself the viewer has a unique chance to observe the events behind the closed doors of a jury room the film begins with the end of the trial the jurors retire to deliberate the case a preliminary vote is taken and the result is 111 in favour of the guilty verdict eleven jurors have raised their hands to convict a young man of killing his father only juror 8 has doubts at first even he does not truly believe the young man to be innocent but notes rightfully that the case for the defence might have been presented in a more convincing manner and that the boy might 

Movie 9
Total user reviews: 50
50
a sprawling western epic that follows the adventures of three gunfighters looking for 200000 in stolen gold sergio leones the good the bad and the ugly is a masterpiece one that continues to get better and better with each viewing  in a way its a morality play weighing the consequences of good and evil but it does so in a realistic manner  sometimes crime does pay at least in the short term and sometimes good does go unrewarded this film probably signaled the death knell of the traditional john wayne white hatblack hat westernthe three main characters make the film  lee van cleef the bad is evil personified  totally ruthless hell do whatever it takes to get what he wants  clint eastwood the good is the man with no name not really good in a traditional sense    but he has a certain sense of honor and tries to do the right thing  watch the scene when he gives a dying confederate soldier a puff of his cigar  powerful and it sums up everything that the man

Movie 12
Total user reviews: 50
50
when i first saw this movie i didnt appreciate it like i do now i think it may have been because i was so young when i first saw it just recently i saw the movie again what an amazing story and moving meaning that movie teaches you so much about life and the meaning of it that life isnt as bad as most people make it seem that an innocent man can impact so many lives with his innocence the meaning of the movie to me is that everyone needs to have a better outlook on life that we need to appreciate more of the little things and not let the big things hold us back that truly although life may throw us trials and tribulations like a box of chocolates but that we have to just bite into it and get through it even if we dont like it that we all need to hold true to our values and not sink into a place that feels like theres no hope i just love this movie and anyone who hasnt seen it or who thinks that dont like it i seriously suggest seeing it or seeing it a

Movie 15
Total user reviews: 50
50
star wars episode v  the empire strikes back 1980 is an excellent timeless classic that it is the best sequel of all time i love this film to death it is my second favorite film and the best sequel in the star wars franchise that i love the empire strikes back is a movie that it is one of the best sequels i can think of it is right there with aliens 1986 it is right there with terminator 2 judgment day 1991 it is an excellent sequel and it really does establish the empire as a value treat empire strikes back is still and excellent film which i really believe is a timeless classic this is my second favorite film from director irvin kershner number 1 favorite irvin kershner film will be robocop 2 1990 which is very underrated action flick i still have to review those movies somedayempire strikes back 1980 is the best one of the series because its deeper darker more philosophical its great because darth vader and the empire are at their most menacing the

...

Movie 27
Total user reviews: 50
50
personally ive never seen anything as original in an animated film as in this deeply mythical fairytale what a surreal idea for a movie its hard to find an adequate description because i also dont want to spoil this in the slightest way but this film has a sense of otherness to it  for lack of a better word  like none ive ever seen and the strange mythical nature of this film  apart from the amazing artwork  is probably one of the main reasons for its appeal to me maybe the themes of the story dont feel quite as strange to an eastern audience because they fit to a certain degree with some easternasian mythologies  to me this beautiful piece of wonder was something new and a profoundly moving experience outstanding animation funny weird scary and touching at the same time this unique work of art is one i cant recommend enough 10 out of 10favorite films httpwwwimdbcomlistmkjokvqlsbslesserknown masterpieces httpwwwimdbcomlistls070242495favorite tvshows r

Movie 33
Total user reviews: 50
50
i saw harakiri seppuku in a new 35 mm print at nycs film forum this is a brilliant use of a narrow period genre to explosively indict politics and culture writers shinobu hashimoto and yasuhiko takiguchi surely must have been as inspired by the count of monte cristo ambrose bierce and howard hawks westerns as much as by samurai literature and moviesthe film begins deceptively as a story within a story seemingly providing a traditional example of upholding samurai honor such as in the conventional oftretold tale of the 47 ronin the context is set at a time when the central government the shogunate is supplanting local clans and arbitrarily unemploying thousands of people notably their samurai forcing them into the mercenary mode of ronin at best and begging for food at worse but the parallels to the 20th century are made repeatedly explicit as the samurai who comes to this clan seeking help is from hiroshimavery gradually we get further insight on the 

Movie 36
Total user reviews: 50
50
what can i say about this movie that was already said it is my favorite time travel scifi adventure epic comedy in the 80s and i love this movie to death when i saw this movie i was thrown out by its theme an excellent scifi adventure epic i love the 80s its simple the greatest time travel movie ever happened in the history of world cinema i love this movie to death i love love love it i firstly saw this movie in the early 90s as a kid on vhs tape and i was blow away by the action and the score and the title back to the future this movie is one of the best masterpiece time travel epic adventure film michael j fox and christopher lloyd are fantastic at acting music theme score by alan silvestri is excellent i could listen to it whole dayback to the future 1985 from the academy award  winning team of steven spielberg and robert zemeckis comes the hilariously groundbreaking adventure that thrilled critics and audiences alike  and sparked one of the most 

Movie 42
Total user reviews: 50
50
if there is one charlie chaplin film to recommend as others have pointed to in the past city lights is the one though chaplin played his tramp character superbly in other movies like modern times and the gold rush city lights displays the tramp at his funniest his bravest his most romantic and his most sympathetic its tough for filmmakers in recent days to bring the audience so close emotionally with the characters but its pulled off the film centers on three characters the tramp the quintessential funny homeless man who blends into the crowd but gets caught in predicaments he helps a drunken businessman myers a fine performance in his own right from suicide and becomes his on and off again friend that is when it suits him and doesnt notice his friends state the other person in the tramps life is the blind flower girl virginia cherrill one of the most absorbing beautiful and key female performances in silent film who are quite fond of each other despi

Movie 49
Total user reviews: 50
50
i mean i was pretty high but stillit definitely lived up to my hype
Movie 50
Total user reviews: 50
50
there are literally hundreds of comments about this movie on imdb  many of them exhort its greatness  i dont disagree with thembut id like to add a suggestion to those of you out there who havent seen this film  id like to tell you how to watch itthe people who made this movie didnt think they were producing a masterpiece bergman left the shoot disgusted  the screenwriters were on salary for warners writing half a dozen movies a year and this was just one more  bogie was punching the clock in the middle of a workhorse careerso as an audience member you cant sit down expecting gilded greatnessdont have a casablaca party  dont watch it on your first date hoping it will lend that romantic touch  dont watch it as part of your i need to watch the best 10 movies of all time film school projectbuy this movie on dvd  have it at the ready  and then one friday

Rotten Tomatoes Reviews: 

In [17]:
url_rot = 'https://www.rottentomatoes.com/top/bestofrt/'
page_rot = requests.get(url_rot, headers = headers)
soup_rot = BeautifulSoup(page_rot.content, "html.parser")

In [18]:
container_rot = soup_rot.find('table', class_ = 'table')
container_rot = container_rot.find_all('a', class_ = 'unstyled articleLink')
print(type(container_rot))

<class 'bs4.element.ResultSet'>


In [19]:
movie_names_rot = []
movies_rot = container_rot[0:50]      #here we get the top 50 movies we want
for movie_rot in movies_rot:
    name_rot = movie_rot.text
    name_sp_rot = name_rot.strip()
    movie_names_rot.append(name_sp_rot)
print(movie_names_rot)

['Black Panther (2018)', 'Avengers: Endgame (2019)', 'Us (2019)', 'Toy Story 4 (2019)', 'Lady Bird (2017)', 'Citizen Kane (1941)', 'Mission: Impossible - Fallout (2018)', 'The Wizard of Oz (1939)', 'BlacKkKlansman (2018)', 'Get Out (2017)', 'The Irishman (2019)', 'Mad Max: Fury Road (2015)', 'Spider-Man: Into the Spider-Verse (2018)', 'Casablanca (1942)', 'Moonlight (2016)', 'A Star Is Born (2018)', 'Wonder Woman (2017)', 'Dunkirk (2017)', 'Inside Out (2015)', 'The Farewell (2019)', 'A Quiet Place (2018)', 'Modern Times (1936)', 'Eighth Grade (2018)', 'It Happened One Night (1934)', 'Booksmart (2019)', 'A Night at the Opera (1935)', 'Roma (2018)', 'The Third Man (1949)', 'Coco (2017)', 'Thor: Ragnarok (2017)', 'The Shape of Water (2017)', 'Spotlight (2015)', 'Selma (2014)', 'The Godfather (1972)', 'La Grande illusion (Grand Illusion) (1938)', 'Arrival (2016)', "Singin' in the Rain (1952)", 'Snow White and the Seven Dwarfs (1937)', 'Logan (2017)', 'The Cabinet of Dr. Caligari (Das Cabin

In [20]:
#now need to click into each movie name in order to see detailed information
#so need to extract all the urls

movie_links_rot = []
movies_rot = container_rot[:50]
for movie_rot in movies_rot:
    #tag = movie
    link_rot = movie_rot.get('href', None)
    movie_links_rot.append(link_rot)
print(movie_links_rot)

['/m/black_panther_2018', '/m/avengers_endgame', '/m/us_2019', '/m/toy_story_4', '/m/lady_bird', '/m/citizen_kane', '/m/mission_impossible_fallout', '/m/the_wizard_of_oz_1939', '/m/blackkklansman', '/m/get_out', '/m/the_irishman', '/m/mad_max_fury_road', '/m/spider_man_into_the_spider_verse', '/m/1003707-casablanca', '/m/moonlight_2016', '/m/a_star_is_born_2018', '/m/wonder_woman_2017', '/m/dunkirk_2017', '/m/inside_out_2015', '/m/the_farewell_2019', '/m/a_quiet_place_2018', '/m/modern_times', '/m/eighth_grade', '/m/it_happened_one_night', '/m/booksmart', '/m/1015002-night_at_the_opera', '/m/roma_2018', '/m/the_third_man', '/m/coco_2017', '/m/thor_ragnarok_2017', '/m/the_shape_of_water_2017', '/m/spotlight_2015', '/m/selma', '/m/godfather', '/m/la_grande_illusion', '/m/arrival_2016', '/m/singin_in_the_rain', '/m/1048445-snow_white_and_the_seven_dwarfs', '/m/logan_2017', '/m/the_cabinet_of_dr_caligari', '/m/double_indemnity', '/m/the_favourite_2018', '/m/the_big_sick', '/m/spider_man_fa

In [21]:
for i in range(len(movie_links_rot)):
    movie_links_rot[i] = 'https://www.rottentomatoes.com'+ movie_links_rot[i]
print(movie_links_rot)

['https://www.rottentomatoes.com/m/black_panther_2018', 'https://www.rottentomatoes.com/m/avengers_endgame', 'https://www.rottentomatoes.com/m/us_2019', 'https://www.rottentomatoes.com/m/toy_story_4', 'https://www.rottentomatoes.com/m/lady_bird', 'https://www.rottentomatoes.com/m/citizen_kane', 'https://www.rottentomatoes.com/m/mission_impossible_fallout', 'https://www.rottentomatoes.com/m/the_wizard_of_oz_1939', 'https://www.rottentomatoes.com/m/blackkklansman', 'https://www.rottentomatoes.com/m/get_out', 'https://www.rottentomatoes.com/m/the_irishman', 'https://www.rottentomatoes.com/m/mad_max_fury_road', 'https://www.rottentomatoes.com/m/spider_man_into_the_spider_verse', 'https://www.rottentomatoes.com/m/1003707-casablanca', 'https://www.rottentomatoes.com/m/moonlight_2016', 'https://www.rottentomatoes.com/m/a_star_is_born_2018', 'https://www.rottentomatoes.com/m/wonder_woman_2017', 'https://www.rottentomatoes.com/m/dunkirk_2017', 'https://www.rottentomatoes.com/m/inside_out_2015',

In [22]:
#found the link of the reviews of each movie
# found the genre of each movie and stored in list format by name #genre_lst_rot

review_links_rot = []
genre_lst_rot = []
for i in movie_links_rot:
    #page = requests.get(i, headers = headers)
    soup_rot = BeautifulSoup(page_rot.content, "html.parser")
    review_link_rot = i + '/reviews?type=user'
    review_links_rot.append(review_link_rot)
    
    #genre
    m_page_rot = requests.get(i, headers = headers)
    m_soup_rot = BeautifulSoup(m_page_rot.content, 'lxml')
    genre_rot = m_soup_rot.find('div', class_ = 'meta-value genre').text.lower().strip()
    genres_rot = re.sub('\s+','',genre_rot).split(',')
    genre_lst_rot.append(genres_rot)
    
print(review_links_rot)
print(genre_lst_rot)

['https://www.rottentomatoes.com/m/black_panther_2018/reviews?type=user', 'https://www.rottentomatoes.com/m/avengers_endgame/reviews?type=user', 'https://www.rottentomatoes.com/m/us_2019/reviews?type=user', 'https://www.rottentomatoes.com/m/toy_story_4/reviews?type=user', 'https://www.rottentomatoes.com/m/lady_bird/reviews?type=user', 'https://www.rottentomatoes.com/m/citizen_kane/reviews?type=user', 'https://www.rottentomatoes.com/m/mission_impossible_fallout/reviews?type=user', 'https://www.rottentomatoes.com/m/the_wizard_of_oz_1939/reviews?type=user', 'https://www.rottentomatoes.com/m/blackkklansman/reviews?type=user', 'https://www.rottentomatoes.com/m/get_out/reviews?type=user', 'https://www.rottentomatoes.com/m/the_irishman/reviews?type=user', 'https://www.rottentomatoes.com/m/mad_max_fury_road/reviews?type=user', 'https://www.rottentomatoes.com/m/spider_man_into_the_spider_verse/reviews?type=user', 'https://www.rottentomatoes.com/m/1003707-casablanca/reviews?type=user', 'https://

In [23]:
# here found the user reviews and the list of words excluding the stopwords in the wordlist_rot

tag_list_rot = []
wordlist_rot = []
n_rot = 0
for i in range(len(review_links_rot)):
    review_page_rot = requests.get(review_links_rot[i], headers = headers)
    soup_rot = BeautifulSoup(review_page_rot.content, "html.parser")
    tag_p_rot = soup_rot.find_all('div', class_ = 'audience-reviews__review-wrap') #find_all to return a list
    tag_rot_f = soup_rot.find('p', class_ = 'audience-reviews__review js-review-text clamp clamp-8 js-clamp')
    tag_rot_r = re.sub("<.?>", "", tag_rot_f.text)
    tag_rot_r = (tag_rot_r.strip()).lower()
    tag_rot = re.sub(r'[^\w\s]', '', tag_rot_r)
    text_token_rot = word_tokenize(tag_rot)
    review3_rot = [word for word in text_token_rot if not word in stopwords.words()]
    n_rot = n_rot + 1
    print('Movie', n_rot)
    top_50_rot = tag_rot[:50]
    print('Total user reviews:', len(top_50_rot))
    print(len(top_50_rot))#some of the movies don't have 50 reviews, but that is fine  
    print(tag_rot)
    tag_list_rot.append(tag_rot)
    wordlist_rot += review3_rot
    
print(wordlist_rot)

Movie 1
Total user reviews: 50
50
it was absolutely appaling i have never been more bored or ashamed of marvel in my entire life dreadful movie would never watch again or recommend
Movie 2
Total user reviews: 50
50
exciting and fun at the sametime you must watch it
Movie 3
Total user reviews: 50
50
i dont know why the movie critics and the tomato meter were so generous with rating this movie i dont particularly rate movie critics views that highly anyway but this was crap thats all i need to say
Movie 4
Total user reviews: 50
50
a heartwarming and funny toy story movie but its not as good as the three films because the characters feel more brave and smarter in the three previous films
Movie 5
Total user reviews: 50
50
meh was ok ronan is always good to watch and doesnt disappoint but if youre not an impoverished quite bright but nongenius glee girl struggling with your identity in a feepaying catholic school in sacramento that your parents cant afford youll probably struggle to relate 

Movie 23
Total user reviews: 50
50
a unique coming of age film set in the modern world awkward but touching
Movie 24
Total user reviews: 50
50
afi 100 greatest films  35 this is a film i knew nothing about and went into this with only the very short description on the tcm listing as background knowledge the brief synopsis under the title listing made this film sound more like a mysterysuspense when i read it so i was surprised and thrown off by the quirky and playful tone early into the picture not realizing i was actually watching one of the original romcoms it isnt hard to see why this was the first film to take home the big four at the academy awards as the directing and acting from the two leads was so in sync and i imagine quite groundbreaking for that time as elements of this film are still seen prevalently in romantic comedies today definitely not my favorite film or genre but one definitely that would be hard not to appreciate and respect
Movie 25
Total user reviews: 50
50
a gr

Movie 47
Total user reviews: 50
50
the dialog in these older films is always pretty good eve was really sketchy from the get go here but they didnt give it enough legs as to what she may be up to the films didnt really pick up till the final act and it was all like blam blackmail deceit back stabbing its like 23 of a movie about one thing and then the final 13 they make it about what they said it was all about  if its all about eve they should have made it all about eve
Movie 48
Total user reviews: 50
50
pretty solid psychological thriller that reminds me of what has macabrely fascinated me since i can remember the more recent real life occurrences where a family member discover their seemingly normal relation is living a double life as a monster no surprise hitchcock recognized and tapped into the horrific implications of this family member being forced to acknowledge the suspicions that because of familial love they are naturally attempting every logistical process to internally dism

In [24]:
# lsit of words from all three websites after excluding the stopwords

print(wordlist_meta)
print(wordlist_imdb)
print(wordlist_rot)

['generally', 'good', 'movie', 'pretty', 'watchable', 'way', 'greatest', 'movie', 'alltime', 'truth', 'probably', 'even', 'top', '1000', 'like', 'babe', 'ruth', 'baseball', 'like', 'john', 'steinbeck', 'writing', 'doubt', 'godfather', 'greatest', 'film', 'time', 'never', 'seen', 'performances', 'stellar', 'marlon', 'brando', 'pacino', 'james', 'caan', 'robertlike', 'babe', 'ruth', 'baseball', 'like', 'john', 'steinbeck', 'writing', 'doubt', 'godfather', 'greatest', 'film', 'time', 'never', 'seen', 'performances', 'stellar', 'marlon', 'brando', 'pacino', 'james', 'caan', 'robert', 'duvall', 'everyone', 'else', 'godfather', 'little', 'life', 'lessons', 'everywhere', 'along', 'engaging', 'plot', 'completes', 'critically', 'acclaimed', 'film', 'time', 'godfather', 'francis', 'ford', 'coppolas', 'masterpiece', 'expand', 'jeff', 'jefferies', 'photographer', 'likes', 'spy', 'lives', 'neighbors', 'rear', 'windows', 'begins', 'suspect', 'killed', 'wife', 'directed', 'alfred', 'hitchcock', 'writ

In [25]:
# created a set of wordlist by combining the wordlist of each website together and by craeting set duplicate values
# are also removed

set_wordlist = set(wordlist_meta)
set_wordlist = set(wordlist_imdb)
set_wordlist = set(wordlist_rot)

print(len(set_wordlist))
print(set_wordlist)

1062
{'towards', 'protagonista', 'work', 'sacramento', 'tiempo', 'dejando', 'decided', 'unfortunately', 'genre', 'glad', 'made', 'deceit', 'britain', 'top', 'three', 'nolan', 'awards', 'believe', 'dancing', 'b', '1700s', 'managing', 'youll', 'toma', 'balance', 'academy', 'encontrará', 'edge', 'resulta', 'impulsar', 'interesante', 'dinero', 'shocking', 'far', 'situaciones', 'especially', 'sobrevivir', 'cumplen', 'deep', 'award', 'complemento', 'scene', 'social', 'wouldnt', 'background', 'animales', 'voz', 'understand', 'keeping', 'amount', 'amor', 'level', 'mysterio', 'revolucionaria', 'times', 'importantes', 'invested', '7510', 'acknowledge', 'maquinas', 'elements', 'cage', 'exciting', 'outstanding', 'incredibly', 'perfectly', 'repeats', 'messaging', 'principales', 'time', 'make', 'protagonizada', 'watching', 'rights', 'cuenta', 'largo', 'views', 'vamos', 'struggling', 'living', 'aristocracy', 'indelible', '35', 'well', 'flashy', 'ride', 'necesitan', 'duke', 'little', 'finger', 'gran',

In [26]:
# created a dictionary of wordlist which shows the frequency of each word, here word is key 
# and its frequency is value 

dic_wordlist = {}
for i in set_wordlist:
    x = wordlist_meta.count(i)
    dic_wordlist[i] = x
    
    x = wordlist_rot.count(i)
    dic_wordlist[i] = x
    
    x = wordlist_imdb.count(i)
    dic_wordlist[i] = x
    
print('Wordfrequency dictionary: ', dic_wordlist)
    

Wordfrequency dictionary:  {'towards': 4, 'protagonista': 0, 'work': 11, 'sacramento': 0, 'tiempo': 0, 'dejando': 0, 'decided': 2, 'unfortunately': 0, 'genre': 6, 'glad': 1, 'made': 18, 'deceit': 0, 'britain': 0, 'top': 5, 'three': 8, 'nolan': 4, 'awards': 2, 'believe': 13, 'dancing': 0, 'b': 0, '1700s': 0, 'managing': 0, 'youll': 3, 'toma': 0, 'balance': 0, 'academy': 4, 'encontrará': 0, 'edge': 4, 'resulta': 0, 'impulsar': 0, 'interesante': 0, 'dinero': 0, 'shocking': 1, 'far': 12, 'situaciones': 0, 'especially': 5, 'sobrevivir': 0, 'cumplen': 0, 'deep': 4, 'award': 4, 'complemento': 0, 'scene': 21, 'social': 7, 'wouldnt': 3, 'background': 4, 'animales': 0, 'voz': 0, 'understand': 14, 'keeping': 2, 'amount': 2, 'amor': 0, 'level': 5, 'mysterio': 0, 'revolucionaria': 0, 'times': 11, 'importantes': 0, 'invested': 0, '7510': 0, 'acknowledge': 0, 'maquinas': 0, 'elements': 4, 'cage': 0, 'exciting': 1, 'outstanding': 1, 'incredibly': 5, 'perfectly': 4, 'repeats': 0, 'messaging': 0, 'princ

Cosine Similarity

In [27]:
print(genre_lst_rot)
print(genre_lst_imdb)
print(genre_lst_meta)

[['fantasy', 'action', 'adventure'], ['fantasy', 'adventure', 'scifi', 'action'], ['horror', 'mysteryandthriller'], ['fantasy', 'kidsandfamily', 'animation', 'adventure', 'comedy'], ['drama', 'comedy'], ['drama'], ['adventure', 'mysteryandthriller', 'action'], ['musical', 'kidsandfamily', 'fantasy'], ['drama', 'comedy', 'crime'], ['horror', 'mysteryandthriller', 'comedy'], ['drama', 'crime'], ['adventure', 'action'], ['fantasy', 'comedy', 'kidsandfamily', 'animation', 'adventure', 'action'], ['drama'], ['drama'], ['drama', 'music', 'romance'], ['adventure', 'action', 'fantasy'], ['drama', 'war', 'history'], ['comedy', 'fantasy', 'kidsandfamily', 'animation'], ['drama', 'comedy'], ['horror', 'mysteryandthriller'], ['comedy'], ['drama', 'comedy'], ['romance'], ['comedy'], ['comedy'], ['drama'], ['mysteryandthriller'], ['music', 'kidsandfamily', 'animation', 'adventure', 'comedy'], ['fantasy', 'comedy', 'scifi', 'adventure', 'action'], ['fantasy', 'romance'], ['drama'], ['drama', 'history

In [28]:
# genre list of rotten tomatoes website combined together into single list

genre_lst_rot_modified = []
for i in range(len(genre_lst_rot)):
  for j in range(len(genre_lst_rot[i])):
    genre_lst_rot_modified.append(genre_lst_rot[i][j])
print(len(genre_lst_rot_modified))
print((genre_lst_rot_modified))

122
['fantasy', 'action', 'adventure', 'fantasy', 'adventure', 'scifi', 'action', 'horror', 'mysteryandthriller', 'fantasy', 'kidsandfamily', 'animation', 'adventure', 'comedy', 'drama', 'comedy', 'drama', 'adventure', 'mysteryandthriller', 'action', 'musical', 'kidsandfamily', 'fantasy', 'drama', 'comedy', 'crime', 'horror', 'mysteryandthriller', 'comedy', 'drama', 'crime', 'adventure', 'action', 'fantasy', 'comedy', 'kidsandfamily', 'animation', 'adventure', 'action', 'drama', 'drama', 'drama', 'music', 'romance', 'adventure', 'action', 'fantasy', 'drama', 'war', 'history', 'comedy', 'fantasy', 'kidsandfamily', 'animation', 'drama', 'comedy', 'horror', 'mysteryandthriller', 'comedy', 'drama', 'comedy', 'romance', 'comedy', 'comedy', 'drama', 'mysteryandthriller', 'music', 'kidsandfamily', 'animation', 'adventure', 'comedy', 'fantasy', 'comedy', 'scifi', 'adventure', 'action', 'fantasy', 'romance', 'drama', 'drama', 'history', 'drama', 'crime', 'drama', 'war', 'drama', 'scifi', 'myste

In [30]:
# genre list of IMDB website combined together into single list

genre_lst_imdb_modified = []
for i in range(len(genre_lst_imdb)):
  for j in range(len(genre_lst_imdb[i])):
    genre_lst_imdb_modified.append(genre_lst_imdb[i][j])

print(genre_lst_imdb_modified)

['drama', 'crime', 'crime', 'action', 'crime', 'biography', 'action', 'crime', 'western', 'action', 'drama', 'drama', 'action', 'action', 'action', 'action', 'biography', 'drama', 'action', 'crime', 'comedy', 'crime', 'crime', 'drama', 'action', 'drama', 'animation', 'crime', 'adventure', 'comedy', 'action', 'crime', 'action', 'animation', 'biography', 'adventure', 'action', 'drama', 'comedy', 'horror', 'action', 'comedy', 'crime', 'biography', 'drama', 'drama', 'animation', 'western', 'biography', 'drama']


In [31]:
# genre list of metacritics website combined together into single list

genre_lst_meta_modified = []
for i in range(len(genre_lst_meta)):
  for j in range(len(genre_lst_meta[i])):
    genre_lst_meta_modified.append(genre_lst_meta[i][j])
print(len(genre_lst_meta_modified))
print(genre_lst_meta_modified)

128
['drama', 'mystery', 'drama', 'thriller', 'crime', 'mystery', 'thriller', 'drama', 'romance', 'war', 'drama', 'drama', 'mystery', 'romance', 'mystery', 'thriller', 'romance', 'drama', 'thriller', 'romance', 'film-noir', 'comedy', 'romance', 'musical', 'drama', 'comedy', 'romance', 'drama', 'drama', 'history', 'fantasy', 'comedy', 'animation', 'family', 'musical', 'drama', 'thriller', 'crime', 'film-noir', 'adventure', 'drama', 'western', 'drama', 'mystery', 'thriller', 'fantasy', 'war', 'comedy', 'romance', 'action', 'adventure', 'mystery', 'thriller', 'drama', 'sport', 'documentary', 'drama', 'mystery', 'crime', 'drama', 'drama', 'romance', 'action', 'adventure', 'western', 'biography', 'drama', 'mystery', 'thriller', 'film-noir', 'drama', 'thriller', 'drama', 'drama', 'history', 'romance', 'war', 'drama', 'mystery', 'thriller', 'horror', 'drama', 'history', 'thriller', 'war', 'drama', 'drama', 'comedy', 'drama', 'animation', 'family', 'musical', 'drama', 'action', 'drama', 'war',

In [32]:
# created a set of genre list by combining the genre list of each website together and by craeting set duplicate 
# values are also removed

set_genre = set(genre_lst_rot_modified)
set_genre = set(genre_lst_imdb_modified)
set_genre = set(genre_lst_meta_modified)

print(len(set_genre))
print(set_genre)

21
{'animation', 'western', 'thriller', 'romance', 'film-noir', 'family', 'crime', 'drama', 'sport', 'war', 'adventure', 'history', 'music', 'action', 'mystery', 'documentary', 'biography', 'fantasy', 'comedy', 'musical', 'horror'}


In [33]:
# created a dictionary of genre which shows the frequency of each genre, here genre is key and its frequency is value,
# this dictionary is created for each website and vector value of each website to calculate cosine similarity

dic_rot = {}
dic_imdb = {}
dic_meta = {}

for i in set_genre:
  x = genre_lst_rot_modified.count(i)
  dic_rot[i] = x

  x = genre_lst_imdb_modified.count(i)
  dic_imdb[i] = x

  x = genre_lst_meta_modified.count(i)
  dic_meta[i] = x
  
lst_dic_rot_item = list(dic_rot.values())
lst_dic_imdb_item = list(dic_imdb.values())
lst_dic_meta_item = list(dic_meta.values())

print('Rotten Tomatoes: ', dic_rot)
print('\nIMDB: ', dic_imdb)
print("\nMetacritics: ", dic_meta)

print('\nRotten Vector: ', lst_dic_rot_item)
print('\nIMDB Vector: ', lst_dic_imdb_item)
print('\nMetacritic Vector: ', lst_dic_meta_item)

Rotten Tomatoes:  {'animation': 5, 'western': 0, 'thriller': 0, 'romance': 4, 'film-noir': 0, 'family': 0, 'crime': 5, 'drama': 21, 'sport': 0, 'war': 2, 'adventure': 14, 'history': 2, 'music': 2, 'action': 10, 'mystery': 0, 'documentary': 0, 'biography': 0, 'fantasy': 13, 'comedy': 18, 'musical': 2, 'horror': 4}

IMDB:  {'animation': 3, 'western': 2, 'thriller': 0, 'romance': 0, 'film-noir': 0, 'family': 0, 'crime': 10, 'drama': 10, 'sport': 0, 'war': 0, 'adventure': 2, 'history': 0, 'music': 0, 'action': 13, 'mystery': 0, 'documentary': 0, 'biography': 5, 'fantasy': 0, 'comedy': 4, 'musical': 0, 'horror': 1}

Metacritics:  {'animation': 3, 'western': 2, 'thriller': 13, 'romance': 11, 'film-noir': 4, 'family': 3, 'crime': 4, 'drama': 39, 'sport': 1, 'war': 5, 'adventure': 3, 'history': 5, 'music': 1, 'action': 3, 'mystery': 10, 'documentary': 1, 'biography': 2, 'fantasy': 3, 'comedy': 9, 'musical': 3, 'horror': 3}

Rotten Vector:  [5, 0, 0, 4, 0, 0, 5, 21, 0, 2, 14, 2, 2, 10, 0, 0, 0,

In [34]:
# calculates the cosine similarity from the above calculated vectors

import math  
def cosine(l1,l2):
  sum = 0
  sum_l1 = 0
  sum_l2 = 0
  for i in range(len(l1)):
    sum += l1[i]*l2[i]
    sum_l1 += l1[i]**2
    sum_l2 += l2[i]**2
  sqrt_sum_l1 = math.sqrt(sum_l1)
  sqrt_sum_l2 = math.sqrt(sum_l2)
  return sum/(sqrt_sum_l1*sqrt_sum_l2)

In [35]:
# printed the co-sine similarity of websites by taking them in pairs

print('Co-sine similarity between Rotten Tomatoes & IMDB')
print(cosine(lst_dic_rot_item,lst_dic_imdb_item))

print('\nCo-sine similarity between IMDB & Metacritics')
print(cosine(lst_dic_imdb_item,lst_dic_meta_item))

print('\nCo-sine similarity between Rotten Tomatoes & Metacritics')
print(cosine(lst_dic_rot_item,lst_dic_meta_item))

Co-sine similarity between Rotten Tomatoes & IMDB
0.675144696256131

Co-sine similarity between IMDB & Metacritics
0.560060911240077

Co-sine similarity between Rotten Tomatoes & Metacritics
0.7170145474344576


In [36]:
# creating a csv file for each website which shows the links, genres and reivews of each movie
# Rotten Tomatoes csv

import pandas as pd

#rotten tomatoes csv
pd.DataFrame({'Links': review_links_rot, 'Genres': genre_lst_rot, 'Reviews':tag_list_rot}).to_csv('Shipra_Review_rot.csv', index=False)
(pd.DataFrame({'Links': review_links_rot, 'Genres': genre_lst_rot, 'Reviews':tag_list_rot}, columns=['Links','Genres','Reviews'])
   .to_csv('Shipra_Review_rot.csv', index=False))



In [37]:
#Metacritics csv

pd.DataFrame({'Links': review_links_meta, 'Genres': genre_lst_meta, 'Reviews':tag_lst_meta}).to_csv('Shipra_Review_meta.csv')
(pd.DataFrame({'Links': review_links_meta, 'Genres': genre_lst_meta, 'Reviews':tag_lst_meta}, columns=['Links','Genres','Reviews'])
   .to_csv('Shipra_Review_meta.csv', index=False))


In [38]:
#IMDB csv

pd.DataFrame({'Links': review_links_imdb, 'Genres': genre_lst_imdb, 'Reviews':tag_list_imdb}).to_csv('Shipra_Review_imdb.csv', index=False)
(pd.DataFrame({'Links': review_links_imdb, 'Genres': genre_lst_imdb, 'Reviews':tag_list_imdb}, columns=['Links','Genres','Reviews'])
   .to_csv('Shipra_Review_imdb.csv', index=False))

Database

In [40]:
# from csv files of each website data is being imported into a database named MovieReviewDatabase, which has 3 tables
# 1 table for each website which stores the links, genres and reviews of each movie.

import sqlite3

conn = sqlite3.connect('MovieReviewDatabase.sqlite')
cur = conn.cursor()
cur.execute('DROP TABLE IF EXISTS MovieReviewTable_Rot')
cur.execute('CREATE TABLE MovieReviewTable_Rot (Links VARCHAR(255), Genre VARCHAR(255), Reviews VARCHAR(255))')
insertcmd_r = 'INSERT INTO MovieReviewTable_Rot (Links, Genre, Reviews) VALUES (?,?,?)'

# for rotten tomatoes table

col_list_r = ["Links", "Genres","Reviews"]
df_r = pd.read_csv("Shipra_Review_rot.csv", usecols=col_list_r)

for i in range(df_r["Links"].size):
    cur.execute(insertcmd_r, (df_r["Links"][i],df_r["Genres"][i],df_r["Reviews"][i]))
    conn.commit()

cur.close()

cur = conn.cursor()
print("MovieReviewTable_Rot: ")
cur.execute('SELECT Links, Genre, Reviews FROM MovieReviewTable_Rot')
for row in cur:
    print(row)
   
#for metacritics table

cur.execute('DROP TABLE IF EXISTS MovieReviewTable_Meta')
cur.execute('CREATE TABLE MovieReviewTable_Meta (Links VARCHAR(255), Genre VARCHAR(255), Reviews VARCHAR(255))')
insertcmd_m = 'INSERT INTO MovieReviewTable_Meta (Links, Genre, Reviews) VALUES (?,?,?)'

col_list_m = ["Links", "Genres","Reviews"]
df_m = pd.read_csv("Shipra_Review_meta.csv", usecols=col_list_m)

for i in range(df_m["Links"].size):
    cur.execute(insertcmd_m, (df_m["Links"][i],df_m["Genres"][i],df_m["Reviews"][i]))
    conn.commit()

cur.close()

cur = conn.cursor()
print("MovieReviewTable_Meta: ")
cur.execute('SELECT Links, Genre, Reviews FROM MovieReviewTable_Meta')
for row in cur:
    print(row)
 
# for IMDB table

cur.execute('DROP TABLE IF EXISTS MovieReviewTable_IMDB')
cur.execute('CREATE TABLE MovieReviewTable_IMDB (Links VARCHAR(255), Genre VARCHAR(255), Reviews VARCHAR(255))')
insertcmd = 'INSERT INTO MovieReviewTable_IMDB (Links, Genre, Reviews) VALUES (?,?,?)'

col_list = ["Links", "Genres","Reviews"]
df = pd.read_csv("Shipra_Review_imdb.csv", usecols=col_list)

for i in range(df["Links"].size):
    cur.execute(insertcmd, (df["Links"][i],df["Genres"][i],df["Reviews"][i]))
    conn.commit()

cur.close()

cur = conn.cursor()
print("MovieReviewTable_IMDB: ")
cur.execute('SELECT Links, Genre, Reviews FROM MovieReviewTable_IMDB')
for row in cur:
    print(row)

cur.close()

MovieReviewTable_Rot: 
('https://www.rottentomatoes.com/m/black_panther_2018/reviews?type=user', "['fantasy', 'action', 'adventure']", 'it was absolutely appaling i have never been more bored or ashamed of marvel in my entire life dreadful movie would never watch again or recommend')
('https://www.rottentomatoes.com/m/avengers_endgame/reviews?type=user', "['fantasy', 'adventure', 'scifi', 'action']", 'exciting and fun at the sametime you must watch it')
('https://www.rottentomatoes.com/m/us_2019/reviews?type=user', "['horror', 'mysteryandthriller']", 'i dont know why the movie critics and the tomato meter were so generous with rating this movie i dont particularly rate movie critics views that highly anyway but this was crap thats all i need to say')
('https://www.rottentomatoes.com/m/toy_story_4/reviews?type=user', "['fantasy', 'kidsandfamily', 'animation', 'adventure', 'comedy']", 'a heartwarming and funny toy story movie but its not as good as the three films because the characters 

MovieReviewTable_Meta: 
('/movie/citizen-kane-1941/user-reviews', "['drama', 'mystery']", 'its generally a good movie and pretty watchable but in no way is it the greatest movie of alltime in truth probably not even in the top 1000')
('/movie/the-godfather/user-reviews', "['drama', 'thriller', 'crime']", 'like babe ruth is to baseball like john steinbeck is to writing their is no doubt about it the godfather is the greatest film of all time never before have we seen performances so stellar by marlon brando al pacino james caan robertlike babe ruth is to baseball like john steinbeck is to writing their is no doubt about it the godfather is the greatest film of all time never before have we seen performances so stellar by marlon brando al pacino james caan robert duvall and everyone else the godfather has little life lessons everywhere along with an engaging plot that just completes the most critically acclaimed film of all time the godfather is francis ford coppolas masterpiece expand')

MovieReviewTable_IMDB: 
('https://www.imdb.com/title/tt0111161/reviews?ref_=tt_urv', "['drama']", 'the shawshank redemption is written and directed by frank darabont it is an adaptation of the stephen king novella rita hayworth and shawshank redemption starring tim robbins and morgan freeman the film portrays the story of andy dufresne robbins a banker who is sentenced to two life sentences at shawshank state prison for apparently murdering his wife and her lover andy finds it tough going but finds solace in the friendship he forms with fellow inmate ellis red redding freeman while things start to pick up when the warden finds andy a prison job more befitting his talents as a banker however the arrival of another inmate is going to vastly change things for all of themthere was no fanfare or bunting put out for the release of the film back in 94 with a title that didnt give much inkling to anyone about what it was about and with columbia pictures unsure how to market it shawshank redemp